# Flatiron School Phase 3 Project

Student name: **Angelo Turri**

Student pace: **self paced**

Project finish date: **?**

Instructor name: **Mark Barbour**

Blog post URL: **?**

# Stakeholder

Trying to identify pumps that need to be fixed / they want to allocate their resources wisely.

# Data Origin & Description

The data was taken from **Kaggle link here**

Descriptions of any of the original variables can be found **below in the dictionary**.

Important variables in the original dataset include:

**list here**

# Rationale & Limitations

Why are you using **method**?

What about the current problem makes linear regression suitable?

Assumptions of our model


Kind of data used

In [502]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, get_scorer_names, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statistics import mode as md
from matplotlib import pyplot as plt
from IPython.display import clear_output, display_html 
from collections import Counter
from pprint import pprint
from itertools import product
import scipy.stats as ss

In [503]:
# Suppresses needless warnings

import warnings
warnings.filterwarnings("ignore")

In [504]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [505]:
testing = pd.read_csv("tanzanian_water_wells/X_test.csv")
X = pd.read_csv("tanzanian_water_wells/X_train.csv")
y = pd.read_csv("tanzanian_water_wells/y_train.csv")['status_group'].map({'functional': 2, 'functional needs repair': 0, 'non functional': 1})

In [506]:
idx = pd.read_csv("tanzanian_water_wells/X_test.csv")['id']

In [507]:
# Random state for any function in the notebook that uses one.
# This is for reproducibility.

state = 42

# Feature Selection

There are quite a few features in this dataset (37). The descriptions of each of these features was taken from Kaggle and listed below. We will not be using all the features, for several reasons:

- Some do not correlate with the target variable (status_group), e.g., the row ID;
- Others have too many categories;
- Some variables differ massively in their values from one dataset to another;
- Others are near copies of different variables in the same dataset, making it pointless to use them.

In [508]:
desc = {'amount_tsh': 'Total static head (amount water available to waterpoint)',
                    'date_recorded': 'The date the row was entered',
                    'funder': 'Who funded the well',
                    'gps_height': 'Altitude of the well',
                    'installer': 'Organization that installed the well', 
                    'id': 'unique identifier of waterpoint',
                    'longitude': 'GPS coordinate',
                    'latitude': 'GPS coordinate',
                    'wpt_name': 'Name of the waterpoint if there is one',
                    'subvillage': 'Geographic location',
                    'region': 'Geographic location',
                    'region_code': 'Geographic location (coded)',
                    'district_code': 'Geographic location (coded)',
                    'lga': 'Geographic location',
                    'ward': 'Geographic location',
                    'population': 'Population around the well',
                    'public_meeting': 'True/False',
                    'recorded_by': 'Group entering this row of data',
                    'scheme_management': 'Who operates the waterpoint',
                    'scheme_name': 'Who operates the waterpoint',
                    'permit': 'If the waterpoint is permitted',
                    'construction_year': 'Year the waterpoint was constructed',
                    'extraction_type': 'The kind of extraction the waterpoint uses',
                    'extraction_type_group': 'The kind of extraction the waterpoint uses',
                    'extraction_type_class': 'The kind of extraction the waterpoint uses',
                    'management': 'How the waterpoint is managed',
                    'management_group': 'How the waterpoint is managed',
                    'payment': 'What the water costs',
                    'payment_type': 'What the water costs',
                    'water_quality': 'The quality of the water',
                    'quality_group': 'The quality of the water',
                    'quantity': 'The quantity of water',
                    'quantity_group': 'The quantity of water',
                    'source': 'The source of the water',
                    'source_type': 'The source of the water',
                    'source_class': 'The source of the water',
                    'waterpoint_type': 'The kind of waterpoint',
                    'waterpoint_type_group': 'The kind of waterpoint'}

### Variable inconsistency across train and test sets

We were given two datasets for analysis – one training dataset, complete with feature and target variables, and a testing dataset, which only had the feature variables (the target values were hidden).

Our goal is to accurately predict the target values in the test dataset. To do this successfully, we must use features in our model that correlate well with the target

Unfortunately, some variables will correlate well with the target in our training dataset, but not so in the test dataset. These variables are categorical, and their categories differ from one dataset to another. In the table below, you see that some categorical variables differ massively in their values from one dataset to another. Therefore, any model that uses these variables will have less success in the test dataset than in the training dataset.

It seems like the following variables should be removed from our features:

- wpt_name
- subvillage
- installer
- funder
- scheme_name
- ward
- date_recorded

In [509]:
differences = []

columns = list(X.select_dtypes(exclude=['float64', 'int64']).columns)

for col in columns:
    difference = set(list(X[col])) ^ set(list(testing[col]))
    differences.append(len(difference))
    
differences_df = pd.DataFrame({'column': list(columns), 'differences': differences}).sort_values(by=['differences'], ascending=False)
differences_df.head(10)

,column,differences
3,wpt_name,43128
5,subvillage,15120
2,installer,1584
1,funder,1403
12,scheme_name,1251
8,ward,145
0,date_recorded,51
14,extraction_type,1
23,quantity,0
21,water_quality,0


In [510]:
X = X.drop(list(differences_df['column'])[:7], axis=1)
testing = testing.drop(list(differences_df['column'])[:7], axis=1)

### Dropping variables that do not correlate with the target column

Two variables, namely "id" and "recorded_by", do not correlate with the target column (status_group).

- **id** is a unique numerical identifier for waterpoints; each waterpoint has a different identifier. An identifier such as this one cannot meaningfully correlate with the target column.

- **recorded_by** has only one value in the entire dataset, and therefore cannot correlate with the target column

In [511]:
X.id

0        69572
1         8776
2        34310
3        67743
4        19728
5         9944
6        19816
7        54551
8        53934
9        46144
10       49056
11       50409
12       36957
13       50495
14       53752
15       61848
16       48451
17       58155
18       34169
19       18274
20       48375
21        6091
22       58500
23       37862
24       51058
25       22308
26       55012
27       20145
28       19685
29       69124
30       46804
31        6696
32       57238
33       12402
34       41583
35       57355
36       67359
37       60048
38       16583
39          25
40       70238
41       12796
42       52019
43       19282
44        1843
45       12958
46       45111
47       13620
48       18864
49       25182
50        3228
51       19308
52       10745
53       32376
54        9417
55       68724
56       74197
57       22961
58       24593
59       31714
60       29952
61       40393
62        2127
63        8265
64       67433
65       51072
66       3

In [512]:
X.recorded_by.value_counts()

recorded_by
GeoData Consultants Ltd    59400
Name: count, dtype: int64

In [513]:
X = X.drop(['id', 'recorded_by'], axis=1)
testing = testing.drop(['id', 'recorded_by'], axis=1)

### Eliminating variables that correlate with each other

We want to eliminate colinearity. I am going to assume that the following groups of variables are strongly correlated with each other, based on the names they were given:

- **region** and **region_code**
- **scheme_management** and **scheme_name**
- **extraction_type**, **extraction_type_group** and **extraction_type_class**
- **management** and **management_group**
- **payment** and **payment_type**
- **water_quality** and **quality_group**
- **quantity** and **quantity_group**
- **source**, **source_type** and **source_class**
- **waterpoint_type** and **waterpoint_type_group**

However, I need to back it up. So I will search amongst the correlations between the categorical variables. While I'm at it, I might as well look at the correlations amongst the numerical variables too.

How do we decide between these variables? I will use the chi-squared test for indepenence to determine how strongly the variables correlate with our target variable, "status_group." Whichever of these variables more strongly correlates with status_group will be used in our models.

In [514]:
# Eliminating null values
X.scheme_management.fillna("None", inplace=True)
X.permit.fillna('Unknown', inplace=True)
X.public_meeting.fillna('Unknown', inplace=True)

# Casting certain features as different dtypes
X['permit'] = X['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['gps_height'] = X['gps_height'].astype('float64')
X['population'] = X['population'].astype('float64')
X['construction_year'] = X['construction_year'].astype('int64')
X['region_code'] = X['region_code'].astype('str')
X['district_code'] = X['district_code'].astype('str')

# !!IMPORTANT!!
# Separating features into numeric and categorical dataframs for preprocessing.
X_cat = X.select_dtypes(exclude=['float64', 'int64'])
X_cat = X_cat.astype('str')
oe = OrdinalEncoder()
oe.fit(X_cat)
X_cat = pd.DataFrame(oe.transform(X_cat), index = X_cat.index, columns = X_cat.columns)

X_numeric = X.select_dtypes(['float64', 'int64'])
mms = MinMaxScaler()
mms.fit(X_numeric)
X_numeric = pd.DataFrame(mms.transform(X_numeric), columns = X_numeric.columns, index = X_numeric.index)

In [515]:
# Eliminating null values
testing.scheme_management.fillna("None", inplace=True)
testing.permit.fillna('Unknown', inplace=True)
testing.public_meeting.fillna('Unknown', inplace=True)

# Casting certain features as different dtypes
testing['permit'] = testing['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
testing['gps_height'] = testing['gps_height'].astype('float64')
testing['population'] = testing['population'].astype('float64')
testing['construction_year'] = testing['construction_year'].astype('int64')
testing['region_code'] = testing['region_code'].astype('str')
testing['district_code'] = testing['district_code'].astype('str')

# !!IMPORTANT!!
# Separating features into numeric and categorical dataframs for preprocessing.
testing_cat = testing.select_dtypes(exclude=['float64', 'int64'])
testing_cat = testing_cat.astype('str')
oe = OrdinalEncoder()
oe.fit(testing_cat)
testing_cat = pd.DataFrame(oe.transform(testing_cat), 
                           index = testing_cat.index, 
                           columns = testing_cat.columns)

testing_numeric = testing.select_dtypes(['float64', 'int64'])
mms = MinMaxScaler()
mms.fit(testing_numeric)
testing_numeric = pd.DataFrame(mms.transform(testing_numeric), 
                               columns = testing_numeric.columns, 
                               index = testing_numeric.index)

# Checking for collinearity among the variables

We can check for collinearity among the numeric variables, but checking amongst the categorical variables is a waste of time.

In [516]:
def cts(val):
    if (val < -0.5):
        color = 'red'
    elif (val > 0.5):
        color = 'red'
    else: 
        color = 'black'
    return 'color: %s' % color

In [517]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.concat([X_numeric, X_cat], axis=1).corr().style.applymap(cts)

,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year,basin,region,region_code,district_code,lga,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
amount_tsh,1.000000,0.076650,0.022134,-0.052670,0.002944,0.016288,0.067915,0.013977,-0.019866,-0.036760,-0.028303,0.010315,-0.000341,0.009852,0.024756,-0.025091,-0.017738,-0.013213,-0.008835,-0.001836,0.013623,-0.118563,0.012624,-0.023995,-0.014526,-0.014526,0.001201,-0.000711,0.034922,-0.043821,-0.042143
gps_height,0.076650,1.000000,0.149155,-0.035751,0.007237,0.135003,0.658727,-0.161768,-0.323833,-0.172466,-0.111196,0.137375,0.025325,0.082152,0.075141,-0.237886,-0.261342,-0.232483,-0.037572,0.025610,0.114219,-0.097431,0.140459,-0.092273,-0.030781,-0.030781,0.121495,0.124323,0.041968,-0.236758,-0.215989
longitude,0.022134,0.149155,1.000000,-0.425802,0.023873,0.086590,0.396732,0.216088,-0.142783,0.268935,0.089039,0.227492,-0.048005,-0.079758,0.097715,-0.027119,-0.018322,0.049411,-0.176484,-0.080633,0.015608,-0.007441,-0.051622,-0.031906,0.015785,0.015785,-0.013228,-0.047479,0.020594,-0.163355,-0.144605
latitude,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.022152,-0.245278,-0.218502,-0.025485,0.078699,-0.049026,-0.268503,0.045487,0.016553,0.073557,-0.005193,-0.000162,0.001696,0.048213,-0.056396,-0.041983,0.145438,-0.010955,0.038672,0.117193,0.117193,0.024544,0.064626,-0.040928,0.160127,0.146218
num_private,0.002944,0.007237,0.023873,0.006837,1.000000,0.003818,0.026056,0.018325,0.037351,0.033997,0.007959,-0.000843,0.002398,0.007336,0.010618,0.017254,0.014590,0.024027,-0.008308,-0.034356,-0.000894,0.008963,-0.001533,-0.006318,0.003544,0.003544,-0.007913,-0.010613,0.008769,-0.021993,-0.019882
population,0.016288,0.135003,0.086590,-0.022152,0.003818,1.000000,0.260910,0.066899,-0.004416,0.146033,-0.054335,-0.004088,0.010163,-0.035422,-0.036358,0.065397,0.060702,0.079631,-0.045170,-0.022093,0.033582,0.034478,-0.031126,0.024627,0.031487,0.031487,-0.089433,-0.090925,0.034772,0.029213,0.016292
construction_year,0.067915,0.658727,0.396732,-0.245278,0.026056,0.260910,1.000000,0.268151,-0.141681,0.376624,-0.132334,0.133283,0.032668,-0.017954,0.080284,-0.061434,-0.056986,0.010066,-0.125290,-0.074380,0.129046,-0.016861,0.050884,-0.008012,-0.008484,-0.008484,-0.007553,-0.025503,0.096427,-0.194506,-0.179239
basin,0.013977,-0.161768,0.216088,-0.218502,0.018325,0.066899,0.268151,1.000000,-0.114880,0.491692,0.009754,-0.006718,0.016602,-0.081049,0.079343,0.167450,0.189307,0.177939,-0.061139,-0.068413,-0.017788,-0.029961,-0.075529,0.044721,-0.026707,-0.026707,-0.061457,-0.074808,0.031636,0.009864,0.013518
region,-0.019866,-0.323833,-0.142783,-0.025485,0.037351,-0.004416,-0.141681,-0.114880,1.000000,0.062788,-0.071604,0.193919,-0.028418,-0.049584,-0.044192,0.217222,0.233877,0.210724,0.076552,-0.023112,-0.054115,0.056709,-0.073382,0.129883,0.034766,0.034766,-0.156225,-0.140439,-0.010899,0.279421,0.257183
region_code,-0.036760,-0.172466,0.268935,0.078699,0.033997,0.146033,0.376624,0.491692,0.062788,1.000000,-0.000508,-0.039177,0.028479,-0.147239,0.113292,0.174353,0.202180,0.256832,-0.124408,-0.131360,-0.034030,0.142661,-0.094935,0.113627,0.057630,0.057630,-0.060097,-0.074246,-0.008932,0.085874,0.087402


In [357]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

pd.reset_option('max_columns')
pd.reset_option('max_rows')

### Strong correlations

These are the strong correlations we observed among our features:

- gps_height and construction_year
- scheme_management, management and management_group
- extraction, extraction_type_group and extraction_type_class
- payment and payment_type
- quantity and quantity_group
- source and source_type
- waterpoint_type and waterpoint_type_group

Of each of these groups, we must pick only one variable to keep.

### Correlation to status_group using chi-squared test of independence

In [358]:
groups = [['gps_height', 'construction_year'], 
          ['scheme_management', 'management', 'management_group'], 
          ['extraction_type', 'extraction_type_group', 'extraction_type_class'], 
          ['payment', 'payment_type'], 
          ['quantity', 'quantity_group'], 
          ['source', 'source_type'], 
          ['waterpoint_type', 'waterpoint_type_group']]

X = pd.concat([X_numeric, X_cat], axis=1)

for group in groups:
    function = SelectKBest(score_func=chi2, k='all').fit(X[group], y)
    function_results = pd.DataFrame({'feature': function.feature_names_in_, 'score': function.scores_}).sort_values(by=['score'], ascending=False).reset_index(drop=True)
    df = function_results.sort_values(by=['score'], ascending=False)
    display(df)

,feature,score
0,gps_height,172.710476
1,construction_year,66.593012


,feature,score
0,scheme_management,310.768525
1,management,176.680250
2,management_group,30.233213


,feature,score
0,extraction_type_class,4962.445269
1,extraction_type_group,3427.761791
2,extraction_type,2638.196579


,feature,score
0,payment,866.203572
1,payment_type,462.559171


,feature,score
0,quantity,672.865229
1,quantity_group,672.865229


,feature,score
0,source_type,568.229788
1,source,458.830986


,feature,score
0,waterpoint_type,3348.517448
1,waterpoint_type_group,2540.881101


This determines which variables we will keep, and which we will eliminate. The ones we will eliminate are:

- construction_year
- management
- management_group
- extraction_type_group
- extraction_type
- payment_type
- quantity_group (this choice doesn't matter, both are perfectly correlated with each other)
- source
- waterpoint_type_group

In [359]:
X_cat = X_cat.drop(['management', 'management_group', 
        'extraction_type_group', 'extraction_type', 'payment_type', 
        'quantity_group', 'source', 'waterpoint_type_group'], axis=1)
X_cat = X_cat.astype('str')
X_cat = pd.get_dummies(X_cat).astype('int64')
X_cat = X_cat.drop(['region_code_26.0'], axis=1)


X_numeric = X_numeric.drop(['construction_year'], axis=1)
scaler = MinMaxScaler()
scaler.fit(X_numeric)
X_numeric = pd.DataFrame(scaler.transform(X_numeric), columns = X_numeric.columns, index = X_numeric.index)

X = pd.concat([X_numeric, X_cat], axis=1)

In [360]:
testing_cat = testing_cat.drop(['management', 'management_group', 
        'extraction_type_group', 'extraction_type', 'payment_type', 
        'quantity_group', 'source', 'waterpoint_type_group'], axis=1)
testing_cat = testing_cat.astype('str')
testing_cat = pd.get_dummies(testing_cat).astype('int64')

testing_numeric = testing_numeric.drop(['construction_year'], axis=1)
scaler = MinMaxScaler()
scaler.fit(testing_numeric)
testing_numeric = pd.DataFrame(scaler.transform(testing_numeric), 
                               columns = testing_numeric.columns, 
                               index = testing_numeric.index)

testing = pd.concat([testing_numeric, testing_cat], axis=1)

# Creating Test and Train Sets

The stakeholder asks us for predictions on the test set, but we do not know what the target values for this test set are (naturally – if we had those, there would be no need for the stakeholder to ask us to do this).

In order to evaluate the performance of our models, we need to create test and train sets from our complete data.

In [361]:
#Creating a train-test-split for X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=state)

In [362]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [363]:
y_test

0        1
1        2
2        2
3        1
4        2
        ..
14845    2
14846    1
14847    0
14848    1
14849    1
Name: status_group, Length: 14850, dtype: int64

# Base Model – Logistic Regression, No Regularization, No Resampling

In [364]:
estimator = LogisticRegression()
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_test)

pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.489960,0.775493,0.728114,0.739327,0.664522,0.729006
recall,0.113594,0.650933,0.884292,0.739327,0.549607,0.739327
f1-score,0.184429,0.707775,0.798639,0.739327,0.563615,0.719475
support,1074.000000,5678.000000,8098.000000,0.739327,14850.000000,14850.000000


# Major Issue – Imbalance Target Categories

As you can see in the value counts of our target variable, the categories are extremely imbalanced. The category of "0", which means the waterpoint has a status of "functional needs repair," has way less values than 1 and 2, which mean "non functional" and "functional," respectively.

We can deal with this situation by introducing new artificial data points of the underrepresented category into our dataset, using SMOTE (Synthetic Minority Oversampling Technique). By default, this function gives each category the same number of occurrences as the most-frequent category, which in this case is "1" - "functional."

In [365]:
y_train.value_counts()

status_group
2    24161
1    17146
0     3243
Name: count, dtype: int64

In [366]:
#Resampling dataframes for model creation
smote = SMOTE(random_state=state)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [367]:
y_train_resampled.value_counts()

status_group
1    24161
2    24161
0    24161
Name: count, dtype: int64

In [368]:
estimator = LogisticRegression()
estimator.fit(X_train_resampled, y_train_resampled)
predictions = estimator.predict(X_test)

pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.234488,0.757632,0.774102,0.673266,0.588741,0.728778
recall,0.573557,0.646883,0.704989,0.673266,0.641809,0.673266
f1-score,0.332883,0.697891,0.737931,0.673266,0.589568,0.693327
support,1074.000000,5678.000000,8098.000000,0.673266,14850.000000,14850.000000


# Analysis – Oversampling Effect

If our model is to be successful, we have to predict the "functional needs repair" instances with respectable accuracy. 13% recall is inexcusable. As we can see with our over-sampling technique, we can improve our accuracy and recall in this category significantly (the f1-score improved from 0.2 to 0.33), but it will slightly decrease our metrics in other categories (functional f1-score decreased from 0.80 to 0.74).

Testing other over-sampling ratios reveals that resampling "functional needs repair" instances to about 10-15,000 instances is ideal for predictive accuracy/recall. Our F1-score for this category tops out at around 0.33-0.34.

This still isn't good, so maybe it's time to use different estimators – e.g., random forests, gradient boosting techniques, etc.

In [369]:
strategies = {str(n): {0: n} for n in [10000, 15000, 20000]}
reports = {}

for name, strategy in strategies.items():
    
    smote = SMOTE(sampling_strategy = strategy, random_state=state)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    estimator =  LogisticRegression()
    estimator.fit(X_train_resampled, y_train_resampled)
    predictions = estimator.predict(X_test)

    report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True))
    reports[name] = report
    
for name, report in reports.items():
    display(report.style.set_caption(f"resampling strategy: {name}"))

,0,1,2,accuracy,macro avg,weighted avg
precision,0.317842,0.782269,0.745770,0.722357,0.615294,0.728776
recall,0.356611,0.634026,0.832798,0.722357,0.607812,0.722357
f1-score,0.336112,0.700389,0.786885,0.722357,0.607796,0.721211
support,1074.000000,5678.000000,8098.000000,0.722357,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.268403,0.787403,0.753242,0.705993,0.603016,0.731239
recall,0.441341,0.627510,0.796122,0.705993,0.621658,0.705993
f1-score,0.333803,0.698422,0.774089,0.705993,0.602105,0.713314
support,1074.000000,5678.000000,8098.000000,0.705993,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.243734,0.794178,0.755292,0.687003,0.597735,0.733163
recall,0.525140,0.610250,0.762287,0.687003,0.632559,0.687003
f1-score,0.332940,0.690170,0.758773,0.687003,0.593961,0.701745
support,1074.000000,5678.000000,8098.000000,0.687003,14850.000000,14850.000000


# Using different estimators

In [370]:
estimators = {'Logistic Regression': LogisticRegression(solver='liblinear', fit_intercept=False, random_state=state), 
              'Decision Tree Classifier': DecisionTreeClassifier(random_state=state), 
              'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3), 
              'Bagging Classifier': BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, max_features=50, random_state=state), 
              'Random Forest': RandomForestClassifier(random_state=state), 
              'XG-Boost': XGBClassifier(random_state=state), 
              'Adaboost': AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=state), 
              'Gradient Boosted Trees': GradientBoostingClassifier(n_estimators=200, max_features=50, random_state=state), 
              'Extra Randomized Trees': ExtraTreesClassifier(n_estimators=100, random_state=state)}

In [371]:
resampled_datasets = {str(n): SMOTE(sampling_strategy=n, random_state=state).fit_resample(X_train, y_train) for n in [{0: 3243}, {0: 10000}, {0: 15000}, {0: 20000}, 'auto']}
resampled_datasets = [(name, key[0], key[1]) for name, key in resampled_datasets.items()]

# results_dict = {}

# for name, estimator in estimators.items():
#     results_dict[name] = [estimator, resampled_datasets]

In [372]:
results_dict = {name: None for name in estimators.keys()}

for name, estimator in estimators.items():
    
    print(f"Estimator: {name}")
    
    results = {}
    
    for ratio, ratio_X, ratio_y in resampled_datasets:
        
        print(f"Sampling ratio: {ratio}")
        
        est = estimator
        est.fit(ratio_X, ratio_y)
        predictions = est.predict(X_test)
        
        report = pd.DataFrame(classification_report(y_test, predictions, output_dict=True))
        matrix = pd.DataFrame(confusion_matrix(y_test, predictions))
        
        sbs = pd.DataFrame({'original': y_test, 'predicted': predictions})
        sbs['correct'] = sbs.apply(lambda row: 1 if row.original == row.predicted else 0, axis=1)
        
        results[ratio] = (predictions, report, matrix, sbs)
        
    results_dict[name] = results
    clear_output(wait=True)

Estimator: Extra Randomized Trees
Sampling ratio: {0: 3243}
Sampling ratio: {0: 10000}
Sampling ratio: {0: 15000}
Sampling ratio: {0: 20000}
Sampling ratio: auto


In [373]:
print("\n")
for name, results in results_dict.items():
    print(color.BOLD + name + color.END)
    
    for ratio, tup in results.items():
        display(tup[1].style.set_caption(f"resampled data (n = {ratio})"))
    print("\n\n")



Logistic Regression


,0,1,2,accuracy,macro avg,weighted avg
precision,0.524038,0.775250,0.728426,0.740673,0.675905,0.731548
recall,0.101490,0.654280,0.886021,0.740673,0.547264,0.740673
f1-score,0.170047,0.709647,0.799532,0.740673,0.559742,0.719637
support,1074.000000,5678.000000,8098.000000,0.740673,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.320205,0.788024,0.745552,0.725185,0.617927,0.731029
recall,0.348231,0.635083,0.838355,0.725185,0.607223,0.725185
f1-score,0.333631,0.703335,0.789235,0.725185,0.608734,0.723440
support,1074.000000,5678.000000,8098.000000,0.725185,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.263866,0.794542,0.752308,0.703838,0.603572,0.733131
recall,0.456238,0.620465,0.795135,0.703838,0.623946,0.703838
f1-score,0.334357,0.696796,0.773128,0.703838,0.601427,0.712209
support,1074.000000,5678.000000,8098.000000,0.703838,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.228420,0.799489,0.756834,0.681818,0.594914,0.734927
recall,0.522346,0.606728,0.755619,0.681818,0.628231,0.681818
f1-score,0.317847,0.689897,0.756226,0.681818,0.587990,0.699159
support,1074.000000,5678.000000,8098.000000,0.681818,14850.000000,14850.000000


...

,0,1,2,accuracy,macro avg,weighted avg
precision,0.437696,0.798205,0.804363,0.778519,0.680088,0.775490
recall,0.389199,0.767700,0.837738,0.778519,0.664879,0.778519
f1-score,0.412026,0.782656,0.820711,0.778519,0.671798,0.776603
support,1074.000000,5678.000000,8098.000000,0.778519,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.395498,0.799485,0.808941,0.770842,0.667975,0.775424
recall,0.458101,0.766115,0.815633,0.770842,0.679950,0.770842
f1-score,0.424504,0.782444,0.812273,0.770842,0.673074,0.772823
support,1074.000000,5678.000000,8098.000000,0.770842,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.391205,0.806260,0.811854,0.772458,0.669773,0.779293
recall,0.480447,0.766643,0.815263,0.772458,0.687451,0.772458
f1-score,0.431258,0.785953,0.813555,0.772458,0.676922,0.775352
support,1074.000000,5678.000000,8098.000000,0.772458,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.373626,0.805082,0.810083,0.768148,0.662930,0.776605
recall,0.474860,0.764530,0.809583,0.768148,0.682991,0.768148
f1-score,0.418204,0.784282,0.809833,0.768148,0.670773,0.771739
support,1074.000000,5678.000000,8098.000000,0.768148,14850.000000,14850.000000


,0,1,2,accuracy,macro avg,weighted avg
precision,0.364347,0.795582,0.813613,0.764310,0.657847,0.774226
recall,0.477654,0.773864,0.795629,0.764310,0.682382,0.764310
f1-score,0.413376,0.784573,0.804520,0.764310,0.667490,0.768604
support,1074.000000,5678.000000,8098.000000,0.764310,14850.000000,14850.000000
